In [2]:
from transformers import SamModel, SamConfig, SamProcessor
import torch
import numpy as np
from google.cloud import storage
from PIL import Image
import os 
from google.cloud import storage
from io import BytesIO

2024-02-24 17:34:19.362739: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 17:34:19.362827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 17:34:19.514561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-24 17:34:19.820200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-24 17:34:23.334140: W tensorflow/compiler/tf2

In [3]:
client = storage.Client()
training_bucket = client.get_bucket('training_images_solar')

def import_data(folder, bucket) -> np.array:
    """
    Import images from specified Google Cloud folder
    Returns a list of arrays
    """

    #get all file names
    file_names = [blob.name for blob in bucket.list_blobs(prefix=folder)]

    #storing all the np.array images
    folder_images = []

    i = 1
    for file in file_names:
        #import each picture from folder
        blob = bucket.blob(file)
        as_bytes = blob.download_as_bytes()
        im = Image.open(BytesIO(as_bytes))
        # #append to list as an array
        folder_images.append(np.asarray(im, dtype=np.uint8))

        #just a counter to see how fast it's moving
        print(f"Completed {i} images")
        i+=1

    return folder_images

In [4]:
test = import_data(folder='google_images', bucket=training_bucket)

Completed 1 images
Completed 2 images
Completed 3 images
Completed 4 images
Completed 5 images
Completed 6 images
Completed 7 images
Completed 8 images
Completed 9 images
Completed 10 images
Completed 11 images
Completed 12 images
Completed 13 images
Completed 14 images
Completed 15 images
Completed 16 images
Completed 17 images
Completed 18 images
Completed 19 images
Completed 20 images
Completed 21 images
Completed 22 images
Completed 23 images
Completed 24 images
Completed 25 images
Completed 26 images
Completed 27 images
Completed 28 images
Completed 29 images
Completed 30 images
Completed 31 images
Completed 32 images
Completed 33 images
Completed 34 images
Completed 35 images
Completed 36 images
Completed 37 images
Completed 38 images
Completed 39 images
Completed 40 images
Completed 41 images
Completed 42 images
Completed 43 images
Completed 44 images
Completed 45 images
Completed 46 images
Completed 47 images
Completed 48 images
Completed 49 images
Completed 50 images
Completed

In [5]:
test_img = test[44]

In [6]:
def load_model():
    """Downloads retrained SAM model weights from google cloud bucket and
    returns model ready to make a prediction"""
    # Load the model configuration
    model_config = SamConfig.from_pretrained("facebook/sam-vit-base")
    # processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
    dir = os.getcwd()
    # Load the model configuration
    # Create an instance of the model architecture with the loaded configuration
    model = SamModel(config=model_config)
    model.load_state_dict(torch.load(f'{dir}/../model/fully_trained.pth', map_location=torch.device('cpu')))
    # device = "cuda" if torch.cuda.is_available() else "cpu"
    # model.to(device)
    return model

In [7]:
model = load_model()

In [8]:
def predict_mask(model, image) -> np.array:
    """Takes in a retrained SAM and a google earth
    satellite image in .png format of any size and outputs a black and white image
    corresponding to rooftop masks. Output size is xxx by xxx."""
    processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
    # image_array = np.asarray(Image.open(image))
    # The google images seem to have a 4th alpha dimension, so we need to
    # cut it off with splicing
    image = image[:, :, :-1]
    array_size = image.shape[0]
    # Higher grid sizes seem to confuse the model and decrease performance
    grid_size = 10
    # Generate grid points which will serve as prompt for SAM
    x = np.linspace(0, array_size-1, grid_size)
    y = np.linspace(0, array_size-1, grid_size)
    # Generate a grid of coordinates
    xv, yv = np.meshgrid(x, y)
    # Convert the numpy arrays to lists
    xv_list = xv.tolist()
    yv_list = yv.tolist()
    # Combine the x and y coordinates into a list of list of lists
    input_points = [[[int(x), int(y)] for x, y in zip(x_row, y_row)] for x_row, y_row in zip(xv_list, yv_list)]
    #We need to reshape our nxn grid to the expected shape of the input_points tensor
    # (batch_size, point_batch_size, num_points_per_image, 2),
    # where the last dimension of 2 represents the x and y coordinates of each point.
    #batch_size: The number of images you're processing at once.
    #point_batch_size: The number of point sets you have for each image.
    #num_points_per_image: The number of points in each set.
    input_points = torch.tensor(input_points).view(1, 1, grid_size*grid_size, 2)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    inputs = processor(image, input_points=input_points, return_tensors="pt")
    # Move the input tensor to the GPU if it's not already there
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.eval()
    # forward pass
    with torch.no_grad():
        outputs = model(**inputs, multimask_output=False)
    # apply sigmoid
    mask_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
    # convert soft mask to hard mask
    mask_prob = mask_prob.cpu().numpy().squeeze()
    mask_prediction = (mask_prob > 0.5).astype(np.uint8)
    return mask_prediction

In [ ]:
predict_mask(model, test_img)